In [1]:
#tensorいじりのテストを行う
import torch
import torch.nn
x = torch.randn(10,10,10)
print(torch.cuda.is_available())
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
from audio_to_multiple_pose_gan.config import get_config
import warnings
warnings.simplefilter('ignore')

True


In [2]:
import pandas as pd
from common.consts import AUDIO_SHAPE
batch_size = 32
train_csv: str = "~/Desktop/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#train_csv: str = "~/AI-Experiments/speechandtext2gesture/Gestures/train.csv"
#df = pd.read_csv(self.args.train_csv)
#cfg = get_config(self.args.config)
df = pd.read_csv(train_csv)
#cfg = get_config(config)
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [3]:
from common.consts import AUDIO_SHAPE
configs = {
    "audio_to_pose": {"num_keypoints": 98, "processor": "audio_to_pose", "flatten": False, "input_shape": [None, AUDIO_SHAPE]},
    "audio_to_pose_inference": {"num_keypoints": 98, "processor": "audio_to_pose_inference", "flatten": False, "input_shape": [None, AUDIO_SHAPE]}
}
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE]}
process_row, decode_pose = get_processor(cfg)

In [4]:
from audio_to_multiple_pose_gan.config import get_config
from audio_to_multiple_pose_gan.dataset import generate_batch, get_processor
#get_modelの実装は任せる
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANS, D_patchgan
from audio_to_multiple_pose_gan.static_model_factory import Audio2PoseGANSTransformer, Audio2PoseGANS_STTransformer
from audio_to_multiple_pose_gan.torch_layers import to_motion_delta, keypoints_to_train, keypoints_regloss
from common.audio_lib import save_audio_sample
from common.audio_repr import raw_repr
from common.consts import RIGHT_BODY_KEYPOINTS, LEFT_BODY_KEYPOINTS, LEFT_HAND_KEYPOINTS, \
    RIGHT_HAND_KEYPOINTS, POSE_SAMPLE_SHAPE, G_SCOPE, D_SCOPE, SR, FRAMES_PER_SAMPLE, AUDIO_SHAPE
from common.evaluation import compute_pck
from common.pose_logic_lib import translate_keypoints, get_sample_output_by_config
from common.pose_plot_lib import save_side_by_side_video, save_video_from_audio_video
from torch import optim
import torch
import pandas as pd
import argparse
from torch import nn

def get_training_keypoints():
    training_keypoints = []
    training_keypoints.extend(RIGHT_BODY_KEYPOINTS)
    training_keypoints.extend(LEFT_BODY_KEYPOINTS)
    for i in range(5):
        training_keypoints.extend(RIGHT_HAND_KEYPOINTS(i))
        training_keypoints.extend(LEFT_HAND_KEYPOINTS(i))
    training_keypoints = sorted(list(set(training_keypoints)))
    return training_keypoints

In [5]:
cfg: dict = {"processor": "audio_to_pose", "input_shape": [None, AUDIO_SHAPE], "new_keypoints":True}
process_row, decode_pose = get_processor(cfg)
row = df.sample(n=1).iloc[0]

In [6]:
import glob
valid_X_path = "Gestures/test_256_all/shelly/audio/*"
valid_Y_path = "Gestures/test_256_all/shelly/npz_pose/*"
valid_X_path = glob.glob(valid_X_path)
valid_Y_path = glob.glob(valid_Y_path)
valid_X_path.sort()
valid_Y_path.sort()

In [7]:
valid_Y_path

['Gestures/test_256_all/shelly/npz_pose/10-00:06:49.042376-00:06:53.246580.npz',
 'Gestures/test_256_all/shelly/npz_pose/10-00:06:49.776443-00:06:53.980647.npz',
 'Gestures/test_256_all/shelly/npz_pose/10-00:06:55.982649-00:07:00.186854.npz',
 'Gestures/test_256_all/shelly/npz_pose/10-00:06:56.716717-00:07:00.920921.npz',
 'Gestures/test_256_all/shelly/npz_pose/10-00:06:57.517518-00:07:01.721722.npz',
 'Gestures/test_256_all/shelly/npz_pose/10-00:07:02.389056-00:07:06.593260.npz',
 'Gestures/test_256_all/shelly/npz_pose/1004-00:24:01.875209-00:24:06.079413.npz',
 'Gestures/test_256_all/shelly/npz_pose/1004-00:24:02.275609-00:24:06.479813.npz',
 'Gestures/test_256_all/shelly/npz_pose/1004-00:24:02.475809-00:24:06.680013.npz',
 'Gestures/test_256_all/shelly/npz_pose/1004-00:24:05.745746-00:24:09.949950.npz',
 'Gestures/test_256_all/shelly/npz_pose/1004-00:24:06.680013-00:24:10.884218.npz',
 'Gestures/test_256_all/shelly/npz_pose/1008-00:24:26.833500-00:24:31.037704.npz',
 'Gestures/test_

In [28]:
valid_X_path[24]

'Gestures/test_256_all/shelly/audio/1039-00:50:07.340674-00:50:11.544878.wav'

In [29]:
valid_Y_path[24]

'Gestures/test_256_all/shelly/npz_pose/1039-00:50:07.340674-00:50:11.544878.npz'

In [10]:
import re
pattern = r'(\d{2}-\d{2}:\d{2}:\d{2}\.\d{6}-\d{2}:\d{2}:\d{2}\.\d{6})\.wav'

# マッチする部分を検索
# マッチする部分を検索
m = re.match(pattern, valid_X_path[0])


In [11]:
m

In [30]:
import numpy as np
np.zeros(shape=(0,67727))

array([], shape=(0, 67727), dtype=float64)

In [60]:
from common.audio_repr import raw_repr
raw_repr(valid_X_path[0],sr = SR)

(array([0.00052766, 0.0007947 , 0.00046725, ..., 0.12812093, 0.1513707 ,
        0.        ], dtype=float32),
 16000)

In [61]:
x = np.load(valid_Y_path[0])

In [59]:
for i in x.values():
    break

In [56]:
i.shape

(64, 2, 52)

In [83]:
arr = next(iter(x.values()))
arr = np.delete(arr, [7,8,9], axis = 2)

In [84]:
arr.shape

(64, 2, 49)

In [86]:
from common.pose_logic_lib import normalize_relative_keypoints, preprocess_to_relative, \
    decode_pose_normalized_keypoints, get_pose, decode_pose_normalized_keypoints_no_scaling, decode_pose_normalized_keypoints_new_keypoints

In [87]:
y = preprocess_to_relative(arr)


In [88]:
normalize_relative_keypoints(y, speaker="shelly").shape

(64, 98)

In [80]:
y[np.newaxis,:,:].shape

(1, 64, 98)